In [1]:
import pandas as pd

In [2]:
# converting this table from wide to long is resource intensive - commenting this out because I saved a transformed table locally


# def read_tbl():
#     df = pd.read_excel("../../../../data/NM/23-6-26/copy_of_ipra_request_po16267_fulldata__sheets-20231220144255.xlsx")
#     return df 

# # Select the columns needed for processing
# columns_to_keep = [
#     'Person Certification #', 'Person First Name', 'Person Middle Name',
#     'Person Last Name', 'Person Suffix', 'Person Gender', 'Person Date of Birth',
#     'Year of Birth', 'Person EEOC Category',
#     'Employment Start Date', 'Employment End Date', 'Employment Appointment Type*',
#     'Employment Employment Type*', 'Employment Title/Rank (Current)',
#     'Employment Status', 'Is Primary Employment', 'Employing Organization Name',
#     'Employing Organization Agency Type'
# ]

# # Start by filtering out the relevant columns
# filtered_df = df[columns_to_keep]

# # Identify the columns with the same employment data that might have a number suffix
# employment_columns = ['Employment Start Date', 'Employment End Date', 
#                       'Employment Appointment Type*', 'Employment Employment Type*', 
#                       'Employment Title/Rank (Current)', 'Employment Status', 
#                       'Is Primary Employment', 'Employing Organization Name', 
#                       'Employing Organization Agency Type']

# # Create an empty DataFrame to store the cleaned data
# cleaned_df = pd.DataFrame()

# # Iterate through the rows and extract each employment stint for each person
# for idx, row in df.iterrows():
#     person_info = row[['Person Certification #', 'Person First Name', 'Person Middle Name',
#                        'Person Last Name', 'Person Suffix', 'Person Gender', 
#                        'Person Date of Birth', 'Year of Birth', 'Person EEOC Category']]

#     for col_base in employment_columns:
#         if pd.notnull(row[col_base]):
#             employment_info = {col_base: row[col_base] for col_base in employment_columns}
#             combined_info = pd.concat([person_info, pd.Series(employment_info)], axis=0)
#             cleaned_df = pd.concat([cleaned_df, combined_info.to_frame().T], ignore_index=True)


In [3]:
def clean_dates(df):
    df.loc[:, "start_date"] = df.start_date.str.replace(r"(\w+):(\w+):(\w+)", "", regex=True)
    df.loc[:, "end_date"] = df.end_date.str.replace(r"(\w+):(\w+):(\w+)", "", regex=True)
    return df 

def clean_race(df):
    df.loc[:, "race"] = df.race.str.replace(r"Caucasian", "white", regex=False)
    return df 

def clean_agency(df):
    df.loc[:, "agency_name"] = (df
                           .agency_name
                           .str.lower()
                           .str.strip()
                           .str.replace(r"dept\.", "department", regex=True)
                           .str.replace(r"&", "and", regex=False)
                           .str.replace(r"\bpd\b", "police department", regex=True)
                           .str.replace(r"(\w+)  (\w+)", r"\1 \2", regex=True)
                           .str.replace(r"mex\.", "mexico", regex=True)

    )
    df = df[~((df.agency_name.str.contains("fire")))]
    return df

def read_tbl():
    # df = pd.read_excel("../../../../data/NM/23-6-26/copy_of_ipra_request_po16267_fulldata__sheets-20231220144255.xlsx")
    df = pd.read_csv("../data/output/NM.csv")

    df = df.rename(columns={"Person Certification #": "person_nbr", 
                           "Person First Name": "first_name", 
                           "Person Middle Name": "middle_name", 
                           "Person Last Name": "last_name", 
                           "Person Suffix": "suffix",
                           "Person Gender": "sex", 
                           "Person EEOC Category": "race",
                           "Year of Birth": "year_of_birth", 
                           "Employment Start Date": "start_date", 
                           "Employment End Date": "end_date", 
                           "Employment Employment Type*": "employment_type", 
                           "Employment Status": "employment_status", 
                           "Employing Organization Name": "agency_name", 
                           "Employing Organization Agency Type": "agency_type", 
                           "Employment Title/Rank (Current)": "rank"})
    
    df = df.fillna("")
    
    df = df[["person_nbr", "first_name", "middle_name", "last_name", "suffix", "sex", 
             "year_of_birth", "race", "start_date", "end_date", "employment_type", 
             "rank", "employment_status", "agency_name", "agency_type" ]]
    return df

df = read_tbl()


df = df.pipe(clean_dates).pipe(clean_race).pipe(clean_agency)
df

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_12045/3571126225.py:27: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/output/NM.csv")


,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type
0,96-0214-P,Jason,,Fank,,Male,1970,white,2017-02-27,,Full Time,Sergeant,Active,acoma police department,Law Enforcement Agency
1,96-0214-P,Jason,,Fank,,Male,1970,white,2017-02-27,,Full Time,Sergeant,Active,acoma police department,Law Enforcement Agency
2,96-0214-P,Jason,,Fank,,Male,1970,white,2017-02-27,,Full Time,Sergeant,Active,acoma police department,Law Enforcement Agency
3,96-0214-P,Jason,,Fank,,Male,1970,white,2017-02-27,,Full Time,Sergeant,Active,acoma police department,Law Enforcement Agency
4,96-0214-P,Jason,,Fank,,Male,1970,white,2017-02-27,,Full Time,Sergeant,Active,acoma police department,Law Enforcement Agency
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88363,17-0197-P,Joseph,A,Morales,,Male,1994,Hispanic,2017-05-24,2017-12-18,,Patrol Officer,Resigned,western new mexico university police department,Law Enforcement Agency
88364,17-0197-P,Joseph,A,Morales,,Male,1994,Hispanic,2017-05-24,2017-12-18,,Patrol Officer,Resigned,western new mexico university police department,Law Enforcement Agency
88365,17-0197-P,Joseph,A,Morales,,Male,1994,Hispanic,2017-05-24,2017-12-18,,Patrol Officer,Resigned,western new mexico university police department,Law Enforcement Agency
88366,17-0197-P,Joseph,A,Morales,,Male,1994,Hispanic,2017-05-24,2017-12-18,,Patrol Officer,Resigned,western new mexico university police department,Law Enforcement Agency


In [4]:
df.to_csv("../data/output/new_mexico_index.csv", index=False)